**Section 1: Using SQL and Python in Snowflake**

1. Add the Sample Movie Rental Database from Planet Data to your Snowflake account using the Snowflake Marketplace.
2. Write a query to return a dataset with film and rental, and payment information. Your dataset should have multiple rows per film, one for each time the film was rented and the amount spend on each rental. Create a dataframe with this information.
3. Create a dataframe from the customer table.

In [ ]:
use database MOVIES_5740;


SELECT
    f.FILM_ID,
    f.TITLE,
    f.DESCRIPTION,
    f.RELEASE_YEAR,
    f.RENTAL_DURATION,
    f.RENTAL_RATE,
    f.LENGTH,
    f.MPAA_RATING,
    f.FULLTEXT,
    r.CUSTOMER_ID,
    r.RENTAL_ID,
    r.RENTAL_DATE,
    r.RETURN_DATE,
    p.CUSTOMER_ID,
    p.PAYMENT_ID,
    p.PAYMENT_DATE,
    p.AMOUNT
FROM FILM f
JOIN INVENTORY i
    ON f.FILM_ID = i.FILM_ID
JOIN RENTAL r
    ON i.INVENTORY_ID = r.INVENTORY_ID
JOIN PAYMENT p
    ON r.RENTAL_ID = p.RENTAL_ID
ORDER BY f.FILM_ID

In [ ]:
rentals_df = rentals.to_pandas()

In [ ]:
use database MOVIES_5740;

SELECT * FROM CUSTOMER;

In [ ]:
customers_df = customers.to_pandas()

**Section 2: Manipulating dataframes**

1. Create a column for customer name that has the first name and last name in the same column.
2. Remove any inactive customers from the dataframe

    *Hint: use the active field*
3. Change the email addresses to be 'joe.person@wustl.edu', but only when their store_id is an even number

    *Hint: use apply to run a function over the dataframe, don't forget to select the correct axis*

In [ ]:
# Create a column name that has the first name and the
# last name in the same column
customers_df['FULL_NAME'] = customers_df['FIRST_NAME'] + ' ' + customers_df['LAST_NAME']

# Remove any inactive customers from the dataframe
customers_df = customers_df[customers_df['ACTIVE'] == 1]

# Change the email addresses to be 'joe.person@wustl.edu'
customers_df['EMAIL'] = customers_df.apply(
        lambda row: "joe.person@wustl.edu" if row["STORE_ID"]%2 == 0 else row['EMAIL'],
        axis=1)

customers_df.head() #check

**Section 3: Visualizations**

1. How much does each customer tend to spend in aggregate?

    *Clarification: You want to first create total spend by customer, then you want to visualize that distribution, each customer being an observation. A box and whisker plot would be a good visualization.*

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# calculate total spend by customer
customer_total_spend = rentals_df.groupby("CUSTOMER_ID")["AMOUNT"].sum().reset_index()
customer_total_spend.rename(columns = {"AMOUNT": "TOTAL_SPEND"}, inplace=True)
customer_total_spend = customer_total_spend.merge(
    customers_df[['CUSTOMER_ID','FULL_NAME']],
    on="CUSTOMER_ID",
    how="left"
)
customer_total_spend = customer_total_spend[['CUSTOMER_ID', 'FULL_NAME', 'TOTAL_SPEND']]

# calculate quartiles for labels
Q1 = customer_total_spend["TOTAL_SPEND"].quantile(0.25)
median = customer_total_spend["TOTAL_SPEND"].quantile(0.5)
Q3 = customer_total_spend["TOTAL_SPEND"].quantile(0.75)

# Create plot
sns.boxplot(y="TOTAL_SPEND", data=customer_total_spend)

# Add labels
plt.text(0, Q1, f'Q1: {round(Q1,2)}', ha='center', va='center', backgroundcolor = 'white', fontsize = 'small')
plt.text(0, median, f'MEDIAN: {round(median,2)}', ha='center', va='center', backgroundcolor = 'white', fontsize = 'small')
plt.text(0, Q3, f'Q3: {round(Q3,2)}', ha='center', va='center', backgroundcolor = 'white', fontsize = 'small')

plt.title("Distribution of Total Spend per Customer")
plt.ylabel("Total Spend ($)")
plt.show();

The median customer spends a total of 98.82 on movie rentals. 50% of the customers (within the innerquartile range) spend between 84.80 and 116.73 total on rentals.

2. What does the distribution of film revenue look like?

    *Clarification: You want to first calculate the revenue by film, you can sum the rental rate for each instance that the film was rented using the dataframe you created in section 1 part 2. A histogram would be a good visualization*

In [ ]:
film_revenue = rentals_df.groupby("FILM_ID")["RENTAL_RATE"].sum().reset_index()
film_revenue.rename(columns = {"RENTAL_RATE": "TOTAL_REVENUE"}, inplace=True)
film_revenue = film_revenue.merge(
    rentals_df[["FILM_ID", "TITLE"]].drop_duplicates(),
    on="FILM_ID",
    how="left"
)
film_revenue = film_revenue[['FILM_ID', 'TITLE', 'TOTAL_REVENUE']]

# Create histogram
plt.figure(figsize = (10,6))
plt.hist(film_revenue['TOTAL_REVENUE'], bins=30, edgecolor = 'black')
plt.title("Distribution of Total Revenue per Film")
plt.xlabel("Total Revenue")
plt.ylabel("Number of Films")
plt.grid(axis='y', alpha=0.75)
plt.show()

The majority of films generate between 0 and 60 in total revenue. After 60, the number of films tapers off, with a few generating up to 140 or higher.

**Section 4: Analysis**

*Clarification: You do not need to use a statistical test to answer the following questions. Please use a visualization and interpret what you see. Note the averages, counts, or sums where applicable, and the interpretation.*

1. On average, is the rental rate the same across movie ratings, treat each film as an observation?

In [ ]:
# first, group the films by film_id and get the rental rate
# use first() because you only want one per row and the rental rate only once
film_rates = rentals_df.groupby(["FILM_ID","TITLE","MPAA_RATING"])["RENTAL_RATE"].first().reset_index()

# then average the rental rate for all films across each rating
avg_by_rating = film_rates.groupby("MPAA_RATING")["RENTAL_RATE"].mean().reset_index()
avg_by_rating.head()

plt.figure(figsize=(8,6))
sns.barplot(data=avg_by_rating, x="MPAA_RATING", y="RENTAL_RATE")
plt.title("Average Rental Rate by MPAA Rating")
plt.ylabel("Average Rental Rate")
plt.xlabel("Rating")
plt.show()

Based on the visualization, there does not appear to be a difference in rental rates between film ratings. Of course a test of statistical significance would be required to provide a definitive answer, but the average rates appear similar across all film ratings.

2. Across the various film ratings, are we observing the same number of movies rented at store 2 and store 1?

In [ ]:
# join the customers table with the rentals table to get the store ID
rentals_with_store = rentals_df.merge(
    customers_df[["CUSTOMER_ID", "STORE_ID"]],
    on="CUSTOMER_ID",
    how="left"
)


# count the rentals per store ID and rating
rental_counts = (
    rentals_with_store
    .groupby(["STORE_ID", "MPAA_RATING"])["RENTAL_ID"]
    .count()
    .reset_index(name="RENTAL_COUNT")
)

plt.figure(figsize=(10,6))
sns.barplot(
    data=rental_counts,
    x="MPAA_RATING",
    y="RENTAL_COUNT",
    hue="STORE_ID"
)

plt.title("Number of Rentals by Movie Rating at Each Store")
plt.xlabel("MPAA Rating")
plt.ylabel("Number of Rentals")
plt.legend(title="MPAA Rating")
plt.show()

Across all film ratings, Store 1 has more movies rented than store 2.

3. On average, do films with a character of a 'robot' generate the same amount of revenue (use rental rate) as films that feature a 'teacher'?

    *Hint: sum the rental rate by film across rentals to get revenue by film*

In [ ]:
# create flags for movies with robots and teachers
rentals_df['HAS_ROBOT'] = rentals_df['DESCRIPTION'].str.contains('robot', case=False, na=False)
rentals_df['HAS_TEACHER'] = rentals_df['DESCRIPTION'].str.contains('teacher', case=False, na=False)

# calculate total revenue per film
# pull in the robot and teacher flags
film_revenue = rentals_df.groupby(['FILM_ID', 'TITLE']).agg({
    'RENTAL_RATE': 'sum',
    'HAS_ROBOT': 'max',    # True if any row for that film has a robot
    'HAS_TEACHER': 'max'   # True if any row for that film has a teacher
}).reset_index()

# find the average for each character type
avg_robot_revenue = film_revenue[film_revenue['HAS_ROBOT']]['RENTAL_RATE'].mean()
avg_teacher_revenue = film_revenue[film_revenue['HAS_TEACHER']]['RENTAL_RATE'].mean()
avg_all = film_revenue['RENTAL_RATE'].mean()

# plot results
revenues = [avg_robot_revenue, avg_teacher_revenue, avg_all]
labels = ['Robot', 'Teacher', 'All Films']

bars = plt.bar(labels, revenues, color=['skyblue', 'lightgreen', 'black'])
plt.ylabel('Average Revenue')
plt.title('Average Revenue by Character Type')

# Add labels on top of bars
plt.bar_label(bars, fmt='%.2f')

plt.show()

It looks like both robot and teacher films generate slightly below average revenues. Without doing a statistical test, I can't say that the difference is or is not significant, but based on my own judgement, there doesn't appear to be a strong difference between revenues generated by the two character types.

**Section 5: Analysis**

Its late 2005, and your boss at the DVD rental company wants to know how effective his customer promotion program was. He tells you, 'I want you to give me some descriptive information about how much the customers spent before and after the program started. Were the spending habits similar? Did they differ? Did the program help or make things worse?'

    1. What is the outcome?
    2. What is the main effect/predictor he wants to understand the impact of?
    3. What is the hypothesis?

Lucky for you, your boss already asked Ted in Bethesda to give you a query for how to get the information.

Plus, the statistician you work with has some suggestions for how to give your boss what he wants. Query the data from your container and put it in a Pandas dataframe. Then follow the statistician's suggestions.

    4. Compute summary statistics and create histrograms of the payment_before and payment_after variables. (Try using describe in pandas)
    5. Compute the correlation between these two variables and create a scatter plot
    6. Compute a variable which is the difference between the amounts spent before and after the program started: payment_after - payment_before
    7. Generate a histogram of the difference and conduct a one-sample t-test
    8. Interpret your results.

1. The outcome for this question is customer spending after the program started. We want to determine if there was significant change in spending habits. This will be measured by the variable PAYMENT_AFTER.
2. The main effect/predictor is customer promotion program. We have a date for the program start (7/1/2005) and we will measure that to determine the effect (or if there is one) of the program.
3. The hypothesis in this case is that customer spending changed after the start of the program. In statistical terms, the null hypothesis will be that the program did not change spending and the alternative hypothesis will be that spending changed after the introduction of the promotion program.

In [ ]:
with b4 as (
    select p.customer_id, sum(p.amount) as Payment_before
    from rental r 
    left outer join payment p on p.rental_id = r.rental_id
    where rental_date < cast('2005-07-01' as timestamp) and
        amount is not null
    group by p.customer_id),
aft as (
    select p.customer_id, sum(p.amount) as Payment_after
    from rental r 
    left outer join payment p on p.rental_id = r.rental_id
    where rental_date >= cast('2005-07-01' as timestamp) and
        amount is not null
    group by p.customer_id
)
select distinct c.customer_id, store_id, first_name, last_name,
    active, payment_before, payment_after
from customer c
left outer join b4 r on r.customer_id = c.customer_id
left outer join aft a on a.customer_id = c.customer_id
where payment_after is not null and payment_before is not null;


In [ ]:
# convert query to dataframe
df = query.to_pandas()

# 4. compute summary statistics and create histogram
summary_stats = df[['PAYMENT_BEFORE', 'PAYMENT_AFTER']].describe()
print(summary_stats)

# create histogram showing the spending habits of customers
plt.figure(figsize=(10,5))
plt.hist(df['PAYMENT_BEFORE'], bins=20, alpha=0.5, label='Before')
plt.hist(df['PAYMENT_AFTER'], bins=20, alpha=0.5, label='After')
plt.xlabel('Payment Amount')
plt.ylabel('Number of Customers')
plt.title('Histogram of Payments Before and After Promotion')
plt.legend()
plt.show()

In [ ]:
# 5. compute the correlation between these two variables
correlation = df['PAYMENT_BEFORE'].corr(df['PAYMENT_AFTER'])
print("Correlation between payments before and after:", correlation)

# create a scatter plot
plt.scatter(df['PAYMENT_BEFORE'], df['PAYMENT_AFTER'], alpha=0.5)
plt.xlabel('Payment Before')
plt.ylabel('Payment After')
plt.title('Scatter plot of Before vs After payments')
plt.show()


In [ ]:
from scipy import stats

# 6. compute the difference between payment before and after
df['DELTA_SPEND'] = df['PAYMENT_AFTER'] - df['PAYMENT_BEFORE']

# 7. generate a histogram of the differences
plt.hist(df['DELTA_SPEND'], bins=20, alpha=0.7)
plt.xlabel('Payment After - Payment Before')
plt.ylabel('Number of Customers')
plt.title('Histogram of Payment Differences')
plt.show()

# conduct a one sample t-test
t_stat, p_value = stats.ttest_1samp(df['DELTA_SPEND'], 0)
print("t-statistic:", t_stat)
print("p-value:", p_value)

From the intial histogram, it appears there was an increase in customer spending before and after the start of the program. There did not appear to be a strong correlation in the payment before and after variables from the scatterplot and the correlation coefficient was 0.02, indicating no strong relationship.

Next I calculated the difference between the payment before and after the program start, stored in the variable DELTA_SPEND. To determine if there was a significant change, we want to compare this result to 0 (no change). The t-stat of 65.5 tells us that the mean of the differences was that many errors away from 0 and the p-value tells us that we can reject our null hypothesis and conclude that the promotion program was in fact successful. 

In other words, there **was** a statistically significant increase in customer spending after the introduction of the customer promotion program.